In [1]:
from google.colab import drive
from google.colab import files
import sys
from os.path import join

#### Initiate Directories ####
ROOT = '/content/drive/'     
PROJ = 'My Drive/Colab Notebooks/Project V6/APS360-2020Summer-Project'
drive.mount(ROOT)

PROJECT_PATH = join(ROOT, PROJ)

sPath = PROJECT_PATH + "/saved/splitData" 
ingPath  = PROJECT_PATH + "/data/working-wheat-data/train"
koPath = PROJECT_PATH + "/data/working-wheat-data/train.csv"
save_path = PROJECT_PATH + "/saved/TrainingRuns"

### import library ###

libPath = PROJECT_PATH + '/model/lib/'
#libPath = join(PROJECT_PATH,"/model/lib/")

sys.path.append(libPath)
#!cat '/content/drive/My Drive/ENGSCI/Year 3/SUMMER 2020/APS360/Project/APS360-2020Summer-Project/code/train_AI_lib.py'
from train_AI_lib import *

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
import cv2
#### Filters we want to use ####
edgeDetect = lambda oImg: cv2.Canny(oImg, 100, 100)
laplacian = lambda pImg: cv2.Laplacian(pImg,cv2.CV_64F)
gradient = lambda gImg: cv2.morphologyEx(gImg, cv2.MORPH_GRADIENT, kernel)
opening = lambda cImg: cv2.morphologyEx(cImg, cv2.MORPH_OPEN, kernel)
bilateral = lambda bImg: cv2.bilateralFilter(bImg,9,75,75)

In [14]:
class CNN(nn.Module):
    def __init__(self, name):
        super(CNN, self).__init__()
        self.name = name

        self.conv1 = nn.Conv2d(1,   15,  6, stride=2)  # n = 1024 -> 510
        self.conv2 = nn.Conv2d(15,  30,  6, stride=2)  # n = 510  -> 253
        self.pool1 = nn.MaxPool2d(3, 2)                # n = 253  -> 126
        self.conv3 = nn.Conv2d(30,  60,  6, stride=2)  # n = 126  -> 61
        self.pool2 = nn.MaxPool2d(4, 2)                # n = 61   -> 29

        self.fc1   = nn.Linear(29*29*60, 20)
        self.fc2   = nn.Linear(20, 1)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = F.relu(self.conv2(x))
        x = self.pool2(F.relu(self.conv3(x)))
        x = x.view(-1, 29*29*60)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return(x)

In [8]:
# make dataloaders
trainLoader, valLoader, testLoader = loadData(batchsize = 32, 
                                                   dictPath  = sPath,
                                                   inPath    = ingPath, mode = 'default', args = {'func': edgeDetect})

In [12]:
torch.cuda.is_available()

True

In [17]:
my_model = CNN(name = "edgeDetect")
use_cuda = True

if use_cuda and torch.cuda.is_available():
  my_model.cuda()
  print('CUDA is available!  Training on GPU ...')
else:
  print('CUDA is not available.  Training on CPU ...')

trainNet(net = my_model,
           data = [trainLoader, valLoader], 
           batchsize = 32, 
           epochNo = 20, 
           lr = 0.001, 
           oPath = save_path, mode ='default')

CUDA is available!  Training on GPU ...


TypeError: ignored